%%<br>
mports

In [1]:
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import lxml
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk

import requests

from bs4 import BeautifulSoup,SoupStrainer
from io import StringIO

In [2]:
soup_url = r'https://ballotpedia.org/State_Legislative_Districts'
# %%

In [3]:
all_districts = []
response = requests.get(soup_url, verify = False)
soup = BeautifulSoup(response.content, 'html.parser')
state_districts = soup.find_all("a", href = True)

In [4]:
h_refs = []
for url in state_districts:
    if 'state legislative districts' in str(url):
        # print(url)
        base = "https://ballotpedia.org/"
        full_url = base + str(url.text).replace(' ',"_")
        h_refs.append(full_url)

print(*h_refs, sep="\n")

In [5]:
for ref in h_refs:
    url = ref
    
    page = requests.get(url)
    
    os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\txt files for troubleshooting')

    # Write the page's text content to a file
    # with open('output_soup_strainer.txt', "w", encoding="utf-8") as f:
    #     f.write(page.text)
    # print(page.content)
    table_strainer = SoupStrainer('table', id='officeholder-table')
    page_soup = BeautifulSoup(page.content, 'html.parser', parse_only=table_strainer)

    # print(page_soup.content)
    # print(type(page_soup))
    districts = page_soup.find_all("a")
    total_districts = []
    # print(page_soup.prettify())
    for d in districts:
        total_districts.append(d.text)
        # print(d.text)
    all_districts.extend(total_districts)

In [7]:
state_intitals = []
for state in all_districts:
    state_match = re.findall(state_pat, state)[0]
    ref_lookup = state_ref.get(str(state_match))
    state_intitals.append(ref_lookup)

In [ ]:
def filter_row(row, string_column):
    # Check conditions using an if-else statement
    if re.search(r'[Hh]ouse|[Rr]epresentative', str(row[string_column])):
        return "House"
    elif re.search
                 
        return True  # Row matches the condition
    else:
        return False  # Row does not match the condition
    
    def determine_chamber(title):
    if 'House' in title:
        return 'House'
    elif 'Senate' in title:
        return 'Senate'
    else:
        return 'Unknown'

# Apply the filtering function row-wise
filtered_df = event_data[event_data.apply(filter_row, axis=1)]

In [8]:
districts_w_intials = pd.DataFrame({'state_abbreviation': state_intitals,'district_string': all_districts})
thi_state_districts = districts_w_intials[districts_w_intials['state_abbreviation'].isin(thi_states)]
thi_state_districts.reset_index(inplace=True, drop=True)
# thi_state_districts['chamber'] = ""
# thi_state_districts['chamber'] = ""
thi_state_districts

,state_abbreviation,district_string
0,AL,Alabama State Senate District 1
1,AL,Alabama State Senate District 2
2,AL,Alabama State Senate District 3
3,AL,Alabama State Senate District 4
4,AL,Alabama State Senate District 5
...,...,...
1993,WV,West Virginia House of Delegates District 96
1994,WV,West Virginia House of Delegates District 97
1995,WV,West Virginia House of Delegates District 98
1996,WV,West Virginia House of Delegates District 99


In [ ]:
#create pk

districts_w_keys, key_duplicates = create_pk(thi_state_districts, 'district_string', )

%%splitting up district numbers from rest of string

In [ ]:

def create_pk(df,column):
    lengths = []
    df.loc[:,'state_code'] = np.nan
    df.loc[:,'chamber_code'] = np.nan
    df.loc[:,'district'] = np.nan
    df.loc[:,'primary_key'] = np.nan
    for i,j in enumerate(df[f'{column}']):
        # print(str(j))
        # print(row)
        district_raw = re.split(r'\s(?=District)', str(j))
        match = re.findall(r'\s\d+', str(district_raw))[0]
        match = match.strip()
        if len(match) == 2:
            district_code = '0' + str(match)
        elif len(match) == 1:
            district_code = '00'+str(match)
        else:
            district_code = str(match)
        district_len = len(match)
        lengths.append(district_len)
        ext_state = df.loc[i,'state_abbreviation']
        state_code = state_coding.get(ext_state)
        if 'house' in str(j).lower():
            chamber_code = '0'
        elif 'senate' in str(j).lower():
            chamber_code = '1'
        else:
            print(f'unknown chamber: {str(j)}')
            break
        
        # display_markdown(f'#### {ext_state} - {chamber_code} - {district_raw}', raw=True)
        key_code = f'{state_code}{chamber_code}{district_code}'
        
        
        df.loc[i,'state_code'] = state_code
        df.loc[i,'chamber_code'] = chamber_code
        df.loc[i,'district'] = match
        df.loc[i,'primary_key'] = key_code
    return df

: 

In [ ]:
#dont touch original

lengths = []
thi_state_districts['state_code'] = np.nan
thi_state_districts['chamber_code'] = np.nan
thi_state_districts['district'] = np.nan
thi_state_districts['primary_key'] = np.nan
for i,j in enumerate(thi_state_districts['district_string']):
    # print(str(j))
    # print(row)
    district_raw = re.split(r'\s(?=District)', str(j))
    match = re.findall(r'\s\d+', str(district_raw))[0]
    match = match.strip()
    if len(match) == 2:
        district_code = '0' + str(match)
    elif len(match) == 1:
        district_code = '00'+str(match)
    else:
        district_code = str(match)
    district_len = len(match)
    lengths.append(district_len)
    ext_state = thi_state_districts.loc[i,'state_abbreviation']
    state_code = state_coding.get(ext_state)
    if 'house' in str(j).lower():
        chamber_code = '0'
    elif 'senate' in str(j).lower():
        chamber_code = '1'
    else:
        print(f'unknown chamber: {str(j)}')
        break
    
    # display_markdown(f'#### {ext_state} - {chamber_code} - {district_raw}', raw=True)
    key_code = f'{state_code}{chamber_code}{district_code}'
    
    
    thi_state_districts.loc[i,'state_code'] = state_code
    thi_state_districts.loc[i,'chamber_code'] = chamber_code
    thi_state_districts.loc[i,'district'] = match
    thi_state_districts.loc[i,'primary_key'] = key_code

In [ ]:
    
    
# %% takes duplicates of primary keys and assigns a seat num

eat num of 00 means there is only one seat (no multi-member districts)

In [ ]:
thi_state_districts['count'] = thi_state_districts['primary_key'].map(thi_state_districts['primary_key'].value_counts())
thi_state_districts['seat_num'] = thi_state_districts.groupby('primary_key').cumcount() + 1
thi_state_districts.loc[thi_state_districts['count'] == 1, 'seat_num'] = 0
thi_state_districts = thi_state_districts.drop(columns='count')
thi_state_districts['seat_num'] = thi_state_districts['seat_num'].apply(lambda x: f'{x:02d}')
thi_state_districts['final_primary_key'] = thi_state_districts['primary_key'] + thi_state_districts['seat_num']
# %%

In [ ]:
# %%

In [ ]:
duplicate = thi_state_districts[thi_state_districts.duplicated('primary_key')]

    # print(str(j))
    # print(district_len)
    

print(max(lengths))

In [ ]:
    # print(line.split('District')[-1])

%%

In [ ]:
print(*all_districts, sep='\n')

In [ ]:
[print(x) for x in thi_state_districts]

    # states.append(str(state_match))
# %%

In [ ]:
states = sorted(list(set(states)))
print(len(states))
print(*states, sep="\n")

In [ ]:
for i,j in enumerate(states):
    code = str(i+1)
    if len(code) == 1:
        code = "0"+code
    df = pd.DataFrame({'state': [j], 'code':[code]})
    print(df.to_string())

%%

In [ ]:
    
    break
    print(page.status_code)
    print(page.content[:500])  # Preview the content
    
    # df_list = pd.read_html(page_soup.prettify())
    # print(len(df_list))
    # print(type(df_list))

In [ ]:
    # print(df)

    # dis_soup = BeautifulSoup(response.content, 'html.parser')
    # ditricts_tags = dis_soup.find_all("td")
    # for d in ditricts_tags:
    #     print(d)
    
	

%%